<a href="https://colab.research.google.com/github/marinarhianna/python-tutorials/blob/main/ORBYTS_python_gaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gaming Strategy Classifer 🎯

In this notebook, we are going to train an AI to predict different gaming strategies.

First, let's import our libraries.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

Imagine you are playing a game where you can make your player do any of these moves at any time:

* Attack ⚔
* Defend 🛡
* Heal ⚕
* Wait 🤚

Depending on which combo of these moves your player does, we can classify your gaming strategy style into different categories.

For example, I have sorted these as:

* Defend + Heal = **Cautious**
* Attack + Attack + Heal = **Aggressive**
* Wait + Heal + Defend = **Passive**

Stick with these for now, and go through the notebook. You can come back and change the moves, strategies, and encoding later, if you want to!

In the cell below, we:

* Encode the moves by assigning a number to each move.
* Define the strategies by setting which moves correspond to which strategy, and then create a dataset of size 10 by randomly shuffling each strategy's moves.
* Prepare our dataset for `torch`.





In [2]:
# ─────────────────────────────────────────────────────────────
# ⚔️ GAME MOVE STRATEGY DATASET
# ─────────────────────────────────────────────────────────────

# Encode the moves:

                # 0=attack
                # 1=defend
                # 2=heal
                # 3=wait

move_vocab = [0, 1, 2, 3]

# Define strategy definitions
strategies = {
    0: lambda: [random.choice([1, 2]) for _ in range(10)],             # cautious
    1: lambda: [random.choice([0, 0, 2]) for _ in range(10)],          # aggressive
    2: lambda: [random.choice([3, 2, 1]) for _ in range(10)],          # passive
}

# Add labels and create training tensors in torch
X_game = []
y_game = []
for label in strategies:
    for _ in range(100):
        X_game.append(strategies[label]())
        y_game.append(label)

X_game = torch.tensor(X_game, dtype=torch.float32).unsqueeze(1)
y_game = torch.tensor(y_game)

Now, let's define a simple CNN:

* 1 convolutional layer
* 1 pooling layer
* 1 fully-connected layer


In [3]:
# ─────────────────────────────────────────────────────────────
# 🧠 CNN MODEL
# ─────────────────────────────────────────────────────────────
class Simple1DCNN(nn.Module):
    def __init__(self, input_len, n_classes):
        super(Simple1DCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 8, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)
        self.fc = nn.Linear(8 * (input_len // 2), n_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

Next, let's train the model:

In [5]:
# ─────────────────────────────────────────────────────────────
# 🚀 TRAIN MODEL
# ─────────────────────────────────────────────────────────────

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_game, y_game, test_size=0.2, stratify=y_game)

# Set up model and define input length and number of classes
model = Simple1DCNN(input_len=10, n_classes=3)

# Set up optimizer and loss function
opt = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

# Create dataset loader
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=16, shuffle=True)

# Train the model
for _ in range(10):
    for xb, yb in train_loader:
        opt.zero_grad()
        pred = model(xb)
        loss = loss_fn(pred, yb)
        loss.backward()
        opt.step()

Now, we are going to test how good our AI is at predicting gaming strategies based off combinations of moves.

* Run the cell below, and you will see a box that prompts you to enter a **10-digit** combination of moves.

There will be a gaming strategy in **red** to try to get the AI to predict.

* Use the box to type in a combination of digits associated with that strategy, and see if the AI is able to predict the strategy correctly.

For every correct prediction the AI makes, the score will update.

* Can you trick the AI into predicting the wrong thing?

In [ ]:
# ─────────────────────────────────────────────────────────────
# 🎯 GAMIFIED CHALLENGE INTERFACE
# ─────────────────────────────────────────────────────────────

# Define class names
class_names = ["Cautious", "Aggressive", "Passive"]

# Start scoring system
score = 0

# Set up interface for testing our AI.
goal_label = random.choice([0, 1, 2])
goal_text = widgets.HTML(value=f"<b>🎯 Challenge:</b> Try to make the AI say: <span style='color:red'>{class_names[goal_label]}</span>")
move_input = widgets.Text(description='Your Moves:', placeholder='10 digits like 0123012301')
submit_button = widgets.Button(description='Submit')
result_box = widgets.Output()


# Function to restart interface and update with scores and messages
def on_submit(b):
    global goal_label, score
    with result_box:
        clear_output()
        move_seq = move_input.value.strip()
        if len(move_seq) != 10 or not all(c in '0123' for c in move_seq):
            print("❌ Invalid input. Enter exactly 10 digits using 0-3.")
            return

        move_vals = [int(c) for c in move_seq]
        x = torch.tensor(move_vals, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        probs = torch.softmax(model(x), dim=1).detach().numpy()[0]
        predicted = int(np.argmax(probs))
        confidence = probs[predicted]

        print(f"🤖 Beep! I think your sequence is.. {class_names[predicted]}! I am ({confidence:.2f}) confident.")

        if predicted == goal_label:
            score += 1
            print("🎉 AI successfully predicted your strategy! Beep boop!")
        else:
            print("🔁 AI predicted incorrectly... Beep... Boop..")

        print(f"🏆 Score: {score}")
        goal_label = random.choice([0, 1, 2])
        goal_text.value = f"<b>🎯 New Challenge:</b> Try to make the AI say: <span style='color:red'>{class_names[goal_label]}</span>"

submit_button.on_click(on_submit)

# Display our game interface
display(widgets.HTML("<h3>🎮 AI Strategy Classifier</h3>"))
display(goal_text, move_input, submit_button, result_box)

Some extra things to try out, if you want!

* **Create your own move/strategy encodings!** Just remember to update the rest of the code accordingly, by reading through and making sure the code will be consistent with your changes -- keep track by commenting what you're doing, and for example updating the number of classes if you create more than 3 strategies. Get creative!

* **Adapt the neural network** to make the AI better at predicting strategies by implementing some of the changes you looked at last week (eg. add another convolutional layer, increase number of epochs, etc.)